# Short Time Fourier Transform (continued)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from numpy import fft
from scipy.fftpack import fftshift 
from IPython.display import Image, Audio
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import stft

## Spectral Leakage

When we take the STFT, we apply the DFT to smaller chunks. However, when we break our signal into smaller chunks (or frames), we get discontinuities at the start and end of the signal. 

The DFT assumes that a signal is periodic and continuous, meaning the 'end' point of the signal joins to the 'beginning' of it. When we crop the signal in the STFT, this is often false, so it creates sudden 'jumps' in the signal input. This creates artifacts in the magnitude spectrum output.


If we use another example with mismatched frequency bins by using a non-integer frequency. We can see the artifacts, especially under the effect of periodization.

We're going to "force" periodization to visualize, but remember the DFT assumes this in the calculation already.

As we saw before, we don't get a perfect picture of the 3.5 Hz frequency. Instead, we get frequency "artifacts" or **spectral leakage** due to the discontinuities caused by the non-integer frequency. This puts the frequency off bin and its energy is spread across the frequency bins. 



## Windowing

To try to combat spectral leakage, we need to resolve the discontinuities. We can "smooth out" our signal after chopping it up by applying a windowing function before taking the DFT. 

"Windowing" reduces the amplitude of the discontinuities at the boundaries of each finite sequence of samples. "Windowing" consists of multiplying the time "slice" by a finite-length window with an amplitude that varies smoothly and gradually toward zero at the edges. (I.e., we are applying an envelope function that is tapered at the ends like a 'fade in' and 'fade out'). 
This makes the endpoints of the waveform "meet" and, therefore, results in a continuous waveform without sharp transitions. 

If the amplitude of the wave is smaller at the beginning and end of the window, then the spurious frequencies will be smaller in magnitude as well.

Let's apply a window to our non-integer signal above and see if we can reduce the spectral leakage.

The result is still not perfect, however, we no longer have the energy spread as widely across bins. (Now it is concentrated around bins 3 and 4)

What happens if we apply a window to a signal that does perfectly fit into our frequency bins?

Unfortunately, when we apply a window, we also get some "smearing". This is also **spectral leakage**.

This is due to the property of multiplication in the time domain equaling convolution in the frequency domain. By multiplying the window by our signal, our signal takes on the spectral qualities of our window.

We want to choose a window that minimizes this leakage, while simultaneously pinpointing as accurately as possible the center frequency component.

In [ ]:
Image('../images/leakage.png',width=500)

Aside: That same "smearing" is happening when we try to analyze an offbin frequency without a specific window.

Without choosing a specific windowing function, when we chop our signal into frames we are applying a rectangular window. If you remember from our discussion about perfect lowpass filters, a rectangular window in the time domain is a sinc function in the frequency domain. By effectively multiplying our signal by a rectangular window, we get convolution in the frequency domain of our signal and a sinc function. That sinc frequency content causes those extra frequencies.

## Windowing Functions

There are several different types of window functions that you can apply depending on the
signal. To understand how a given window affects the frequency spectrum, we need to
understand more about the frequency characteristics of windows.

An actual plot of a window shows that the frequency characteristic of the window itself (as a signal) is a continuous
spectrum with a **main lobe** and **side lobes**. 
The main lobe will be centered at each frequency
component of the time-domain signal, and the side lobes approach zero. The height of the side
lobes indicates the affect the windowing function has on frequencies around main lobes. 

If we think of this window as an input signal, let's examine it's spectral content in isolation:

In [ ]:
#take fft of window
N = 128 # window size in # of samples
pad = 8192
Xw = np.fft.fft(window, pad)
Xw2 = np.fft.fft(window2, pad)
Xw3 = np.fft.fft(window3, pad)
Xw4 = np.fft.fft(window4, pad)
ns = np.arange(-N*pad/2,N*pad/2) #center the frequencies for positive and negative components
response = np.abs(fftshift(Xw))# plot of centered response of window -see fftshift docs-
response2 = np.abs(fftshift(Xw2))
response3 = np.abs(fftshift(Xw3))
response4 = np.abs(fftshift(Xw4))
freq = np.fft.fftfreq(pad, d=1)   # normalized frequency
freq = np.fft.fftshift(freq) 
plt.plot(freq, response, freq, response2, freq, response3, freq, response4)
plt.legend(['hanning','hamming','blackmann-harris', 'rectangle'])

Most commonly if you read about windows and their spectral properties, it will be shown on a logarithmic scale instead of linear. Let's look at the same window on a log scale:

In [ ]:
#plot on log scale to better show side lobes
X = np.abs(response)
X2 = np.abs(response2)
X3 = np.abs(response3)
X4 = np.abs(response4)

plt.plot(freq,20*np.log10(X/np.max(X)+1e-10))
plt.plot(freq,20*np.log10(X2/np.max(X2)+1e-10))
plt.plot(freq,20*np.log10(X3/np.max(X3)+1e-10))
plt.plot(freq,20*np.log10(X4/np.max(X4)+1e-10))
plt.legend(['hanning','hamming','blackmann-harris', 'rectangle'])
#can add a tiny number to avoid error that happens when taking log of zero...

In addition, frequently you will see the log spectrum is normalized (so magnitude is divided by max magnitude):

In [ ]:
#plot on log scale; normalized magnitude - in this case it looks very similar.
plt.plot(freq,20*np.log10(response/np.max(response)+ 1e-10))
plt.plot(freq,20*np.log10(response/np.max(response2)+ 1e-10))
plt.plot(freq,20*np.log10(response3/np.max(response3)+ 1e-10))
plt.plot(freq,20*np.log10(response4/np.max(response4)+ 1e-10))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(freq,20*np.log10(response/np.max(response)+ 1e-10))
plt.plot(freq,20*np.log10(response/np.max(response2)+ 1e-10))
plt.plot(freq,20*np.log10(response3/np.max(response3)+ 1e-10))
# plt.plot(freq,20*np.log10(response4/np.max(response4)+ 1e-10))
plt.ylim([-120, 5])
plt.title("Window Spectra (dB)")
plt.xlabel("Normalized Frequency")
plt.ylabel("Magnitude (dB)")
plt.legend(['hanning','hamming','blackmann-harris', 'rectangle'])

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(freq,20*np.log10(response/np.max(response)+ 1e-10))
plt.plot(freq,20*np.log10(response/np.max(response2)+ 1e-10))
plt.plot(freq,20*np.log10(response3/np.max(response3)+ 1e-10))
plt.plot(freq,20*np.log10(response4/np.max(response4)+ 1e-10))
plt.ylim([-120, 5])
plt.xlim(-0.2, 0.2)
plt.title("Window Spectra (dB)")
plt.xlabel("Normalized Frequency")
plt.ylabel("Magnitude (dB)")
plt.legend(['hanning','hamming','blackmann-harris', 'rectangle'])

The ideal is to have narrowest possible main lobe, and lowest side-lobe level. 

Window features:  

**Rectangular window**
* main-lobe width: 2 bins (the minimum/ideal); side lobe level ~ -13db, moderate roll-off

**Hamming window**
* main-lobe width: 4 bins; side lobe level ~ -42db, low roll-off

**Hanning window**
* main-lobe width: 4 bins; side lobe level ~ -31db, high roll-off

**Four term Blackman-Harris window**
* main-lobe width: 8 bins; side lobe level ~ -92db! (below noise floor - we will not hear them), moderate-high roll-off


Typically, lower side lobes reduce leakage in the measured STFT but
increase the bandwidth of the major lobe. The side lobe roll-off rate is the asymptotic decay rate
of the side lobe peaks. By increasing the side lobe roll-off rate, you can reduce spectral leakage. However, by increasing the width of the main lobe, you reduce the spectral accuracy by 'spreading' the energy across multiple bins.

Thus, if the amplitude accuracy of a single frequency component is more important than the
exact location of the component in a given frequency bin, choose a window with a wide
main lobe.  


In general, the Hanning (Hann) window is satisfactory in 95 percent of cases. It has good
frequency resolution and reduced spectral leakage. If you do not know the nature of the
signal but you want to apply a smoothing window, start with the Hanning window.


In [ ]:
from IPython.display import Image
Image('../images/window_functions.gif', embed=True)

### Hop size

When we apply a window to a portion of the signal, we necessarily remove content. Thus, if we align windows "end to end" there will be significant data loss. In order to minimize this data loss, we "hop" the frame (advance the window) by a fraction of its total length (usually around 50-75%). This way, when we add across the frames, we "get back" the amplitudes of the frequency components lost due to windowing.

A good rule of thum is to use 
- 50\% overlap for the rectangular window, 
- 75\% overlap for Hamming and Hanning windows
- and 83\% (5/6) overlap for Blackman windows. 

In [ ]:
Image('../images/hopping2.png')

### STFT and zero-padding

Sometimes we will use a segment of audio (frame) that is relatively short. We now understand that while this allows better resolution in the time domain (seeing 'when' things happen), this compromises the frequency resolution (i.e., 'where' things happen). One 'trick' to gain increased frequency resolution for shorter slice of time is to use a technique called **zero padding**.

With zero-padding, we simply add a bunch of empty (zero-valued) samples to the end of our slice of audio. This, of course, increases the total length of the segment, giving increased frequency resolution. However, since the numbers added are all zeros, it does not harm or change the frequency content, since anything multiplied by zero will be zero.

Zero-padding helps increase frequency resolution because it gives us more points/samples of *k* frequency bins. The spectral content and output envelope is the same, but it creates a smoother version because of more sampling points.

It creates an interpolated version of the frequency domain. 

Remember, that by zero-padding we are increasing the DFT frame length which gives us more, closely spaced bins. This will help us better identify individual (initially) off-bin frequencies, but it won't help us resolve frequencies that are adjacent or in shared bins. It doesn't change the fundamental frequency resolution, just interpolates the initial result across more data points making it smoother.

We can also see in this example that we did not apply a window. So by adding more data points (frequency bins), we showed off the rectangular window/sinc function clearer.

### Fast Fourier Transform (FFT)

The usefulness of the DFT was extended greatly when a fast version was invented by Cooley and Tukey in 1965. This implementation, called the *fast Fourier transform* or FFT, reduces the computational complexity 

from $\mathcal{O}(N^2)$ to $\mathcal{O}(N\ log_2(N))$

The FFT is efficient because redundant or unnecessary computations are eliminated. For example, there's no need to perform a multiplication with a term that contains sin(0) or cos(0).

In addition, to take advantages of certain symmetrical properties of the DFT, the FFT algorithm has to operate on blocks of samples where the number of samples is a power of 2. 

### FFT in `numpy` and `scipy` packages

Both `numpy` and `scipy` have FFT functions. They are very similar. Since we'll mostly be working now with STFTs, it will be important for you to understand the parameters behind your STFT function, and that the calculation will make use of the FFT (not DFT)  which is why the window sizes are always powers of two. 

E.g., in the `numpy.fft.rfft` function it takes in `n` as the first argument 

`n: Number of points along transformation axis in the input to use. If n is smaller than the length of the input, the input is cropped. If it is larger, the input is padded with zeros. If n is not given, the length of the input along the axis specified by axis is used.`



Here's a handy function to find the next closest power of two...

In [ ]:
### use zero-padding to improve frequency resolution:
import math
def NextPowerOfTwo(number):
    # Returns next power of two following 'number'
    return 2**(math.ceil(np.log2(number)))

By default the `rfft` (fft for real-valued input) implementation will zero-pad a signal to this next power of two (or to the FFT size specified).

Zero padding results in a 'smoother' interpretation of the magnitude components.

In [ ]:
Audio('../audio/soprano-E4.wav')

In [ ]:
(fs, data) = read('../audio/soprano-E4.wav')

M1 = 520 # window size
s1 = data[5000:5000+M1] * np.hamming(M1)  #single slice of data windowed
pad = np.zeros(NextPowerOfTwo(M1))
pad[:M1] = s1 # padded signal
s2 = pad
#s2 = np.concatenate((s1, pad)) # padded signal
X1 = rfft(s1)
X2 = rfft(s2)

fig, axs = plt.subplots(2, figsize=(11,5))


axs[0].plot(20*np.log10(abs(X1)/np.max(abs(X1)))) # change y-axis to db power spectrum to see more detail
axs[0].set_title('magnitude spectrum for s1')#x-axis values are bin indices, not frequencies
axs[1].plot(20*np.log10(abs(X2)/np.max(abs(X2))))
axs[1].set_title('magnitude spectrum for s2')
plt.tight_layout();



In [ ]:
window = signal.hann(526) #vary between 51 and 2048
plt.figure()
A = np.fft.rfft(window, 1028) # recall rfft is automatically zero-padding here...
freq = np.linspace(-(len(A)/2),len(A)/2, len(A))
mag = np.abs(A)
response = 20 * np.log10(mag/np.max(mag)) #normalize amplitudes convert to dB
plt.plot(freq, response)
plt.title("Frequency response of the Hann window")
plt.ylabel("Normalized magnitude [dB]")
plt.xlabel("Normalized frequency [cycles per sample]")

## Plotting Spectrogram

There are several functions where you can simply pass the raw audio data and have the function compute the STFT and resulting spectrogram for you. However, it is much better if you understand what you are doing, and plot the spectrogram yourself.

Matplotlib has a plotting tool called `pcolormesh` which can help us do exactly this. (see [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html) for documentation.)

In [ ]:
#read audio
(fs, data) = read('../audio/soprano-E4.wav')
#compute STFT
#Function args:
'''scipy.signal.stft(x, fs=1.0, window='hann', nperseg=256, noverlap=None, 
nfft=None, detrend=False, return_onesided=True, boundary='zeros', padded=True, 
axis=- 1)'''
#Note default overlap 'None' actually is 50% - see docs.
f,t,Z = stft(data, fs=fs)
(numbins,numspectra) = Z.shape;
Z.shape


In [ ]:
plt.pcolormesh(t,f,20*np.log10(np.abs(Z)))
#plt.yscale('log')
#plt.ylim(10,10**4)

### ISTFT
Like the DFT, the STFT has the inverse transform. This way we can recover the original sound from the amplitude and phase spectrogram. Just like the DFT, where we overlap and add their spectra, now we we take inverse DFT of the spectral representation of every frame, and we then shift and sum ("overlap add") adding over the previous frame in such a way that we compensate for the windowing and we recover the output signal.

There are many properties to the STFT, many of which can make the ISTFT *not* invertible.  In short, depending on the frequency resolution of the STFT, it may become non-invertible.

It is important to understand that spectrogram data is not STFT data since we are only plotting magnitude and not phase. Unlike the output of STFT, the spectrogram contains no (direct) phase information. For this reason, it is not possible to reverse the process and generate a copy of the original signal from a magnitude spectrogram alone.